In [7]:

class args_init:
    def __init__(self):
        self.model_path = "./FastSAM.pt"
        self.img_path = "./images/imageForSegmentation.png"
        self.imgsz = 1024
        self.iou = 0.9
        self.text_prompt = None

        self.conf = 0.4
        self.output = "./output/"
        self.point_prompt="[[0,0]]"

        self.randomcolor = True


        self.point_label = "[0]"
        self.box_prompt = "[[0,0,0,0]]"
        self.better_quality = False
        self.device = torch.device(
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        self.retina = True
        self.withContours = False

args = args_init()

In [8]:
args.model_path

'./FastSAM.pt'

In [9]:
# load model
model = FastSAM(args.model_path)
args.point_prompt = ast.literal_eval(args.point_prompt)
args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
args.point_label = ast.literal_eval(args.point_label)
input = Image.open(args.img_path)
input = input.convert("RGB")
everything_results = model(
    input,
    device=args.device,
    retina_masks=args.retina,
    imgsz=args.imgsz,
    conf=args.conf,
    iou=args.iou    
    )
bboxes = None
points = None
point_label = None
prompt_process = FastSAMPrompt(input, everything_results, device=args.device)
if args.box_prompt[0][2] != 0 and args.box_prompt[0][3] != 0:
        ann = prompt_process.box_prompt(bboxes=args.box_prompt)
        bboxes = args.box_prompt
elif args.text_prompt != None:
    ann = prompt_process.text_prompt(text=args.text_prompt)
elif args.point_prompt[0] != [0, 0]:
    ann = prompt_process.point_prompt(
        points=args.point_prompt, pointlabel=args.point_label
    )
    points = args.point_prompt
    point_label = args.point_label
else:
    ann = prompt_process.everything_prompt()
prompt_process.plot(
    annotations=ann,
    output_path=args.output+args.img_path.split("/")[-1],
    bboxes = bboxes,
    points = points,
    point_label = point_label,
    withContours=args.withContours,
    better_quality=args.better_quality,
)




0: 768x1024 79 objects, 230.5ms
Speed: 4.4ms preprocess, 230.5ms inference, 254.8ms postprocess per image at shape (1, 3, 1024, 1024)


In [10]:
prompt_process

In [13]:
from PIL import Image
def sm(im):
    Image.fromarray(im).show()

In [14]:
sm(prompt_process.plot_to_result(
        ann, 
        bboxes, 
        points, 
        point_label, 
        True,
        args.better_quality, 
        args.retina, 
        args.withContours,
    ))

